# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,6.17,74,8,10.22,TF,1707716016
1,1,bajina basta,43.9708,19.5675,7.80,86,99,1.63,RS,1707716016
2,2,hadibu,12.6500,54.0333,26.53,65,24,2.85,YE,1707716016
3,3,grytviken,-54.2811,-36.5092,4.47,78,12,4.46,GS,1707716017
4,4,waitangi,-43.9535,-176.5597,13.79,82,88,2.68,NZ,1707716017


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
coordinates= pd.DataFrame(city_data_df)
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df= city_data_df.loc[(city_data_df['Max Temp']>22)]
city_data_df= city_data_df.loc[(city_data_df['Max Temp']<27)]
city_data_df= city_data_df.loc[(city_data_df['Humidity']>50)]
city_data_df= city_data_df.loc[(city_data_df['Cloudiness']<50)]
city_data_df= city_data_df.loc[(city_data_df['Wind Speed']<3)]
# Drop any rows with null values
city_data_df= city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,hadibu,12.6500,54.0333,26.53,65,24,2.85,YE,1707716016
50,50,puerto ayora,-0.7393,-90.3518,26.97,96,15,2.24,EC,1707716022
113,113,kailua-kona,19.6406,-155.9956,23.98,80,40,2.57,US,1707715919
143,143,haiku-pauwela,20.9219,-156.3051,22.36,83,20,2.06,US,1707716003
167,167,port saint john's,-31.6229,29.5448,23.37,70,22,2.39,ZA,1707716037
180,180,bechem,7.0903,-2.0250,22.32,99,27,0.63,GH,1707716038
194,194,la'ie,21.6477,-157.9253,23.98,79,0,2.22,US,1707716040
196,196,wailua homesteads,22.0669,-159.3780,24.24,81,0,2.06,US,1707716040
199,199,prado,-17.3411,-39.2208,24.22,90,15,1.39,BR,1707715848
216,216,brisas de zicatela,15.8369,-97.0419,24.69,80,10,2.37,MX,1707716043


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= city_data_df[["City", "Lat", "Lng", "Country", "Humidity"]].copy()

hotel_df =city_data_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,hadibu,12.6500,54.0333,26.53,65,24,2.85,YE,1707716016,
50,50,puerto ayora,-0.7393,-90.3518,26.97,96,15,2.24,EC,1707716022,
113,113,kailua-kona,19.6406,-155.9956,23.98,80,40,2.57,US,1707715919,
143,143,haiku-pauwela,20.9219,-156.3051,22.36,83,20,2.06,US,1707716003,
167,167,port saint john's,-31.6229,29.5448,23.37,70,22,2.39,ZA,1707716037,
180,180,bechem,7.0903,-2.0250,22.32,99,27,0.63,GH,1707716038,
194,194,la'ie,21.6477,-157.9253,23.98,79,0,2.22,US,1707716040,
196,196,wailua homesteads,22.0669,-159.3780,24.24,81,0,2.06,US,1707716040,
199,199,prado,-17.3411,-39.2208,24.22,90,15,1.39,BR,1707715848,
216,216,brisas de zicatela,15.8369,-97.0419,24.69,80,10,2.37,MX,1707716043,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"hotel",
    "limit":20,
    "apikey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row['Lat']
    longitude= row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places" + geoapify_key


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hadibu - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
port saint john's - nearest hotel: No hotel found
bechem - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
prado - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found
montevideo - nearest hotel: No hotel found
plettenberg bay - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
guarapari - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
rafaela - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
maceio - nearest hotel: No hotel found
nioki - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
sabana de torres - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,hadibu,12.6500,54.0333,26.53,65,24,2.85,YE,1707716016,No hotel found
50,50,puerto ayora,-0.7393,-90.3518,26.97,96,15,2.24,EC,1707716022,No hotel found
113,113,kailua-kona,19.6406,-155.9956,23.98,80,40,2.57,US,1707715919,No hotel found
143,143,haiku-pauwela,20.9219,-156.3051,22.36,83,20,2.06,US,1707716003,No hotel found
167,167,port saint john's,-31.6229,29.5448,23.37,70,22,2.39,ZA,1707716037,No hotel found
180,180,bechem,7.0903,-2.0250,22.32,99,27,0.63,GH,1707716038,No hotel found
194,194,la'ie,21.6477,-157.9253,23.98,79,0,2.22,US,1707716040,No hotel found
196,196,wailua homesteads,22.0669,-159.3780,24.24,81,0,2.06,US,1707716040,No hotel found
199,199,prado,-17.3411,-39.2208,24.22,90,15,1.39,BR,1707715848,No hotel found
216,216,brisas de zicatela,15.8369,-97.0419,24.69,80,10,2.37,MX,1707716043,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
#coordinates= pd.DataFrame(hotel_df)
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size ="Humidity",
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)